# Code to check dimensions of images



In [ ]:
from PIL import Image
import os
import csv

# function to use in sort of simple list
def sort_funct(row):
    return row

# write a list of dictionaries to a CSV file
def write_dicts_to_csv(table, filename, fieldnames):
    with open(filename, 'w', newline='', encoding='utf-8') as csv_file_object:
        writer = csv.DictWriter(csv_file_object, fieldnames=fieldnames)
        writer.writeheader()
        for row in table:
            writer.writerow(row)


In [ ]:

working_directory = os.getcwd()
image_dir = working_directory + '/gallery_images/'
print(image_dir)
image_names = os.listdir(image_dir)
image_names.sort(key = sort_funct)
image_names.remove('.DS_Store')
image_names.remove('misoriented')

In [ ]:
images = []
for image_name in image_names:
    image = {}
    image['name'] = image_name
    year = image_name[0:4]
    rest = image_name[5:]
    rest = rest[:-4] # remove the ".jpg" from the end
    accession = year + '.' + rest
    #print(image_name, accession)
    image['accession'] = accession
    
    image_path = image_dir + image_name
    with Image.open(image_path) as img:
        width, height = img.size
    #print(height, width)
    #print()
    image['height'] = height
    image['width'] = width
    images.append(image)

fieldnames = list(images[0].keys())
write_dicts_to_csv(images, 'image_dimensions.csv', fieldnames)

print('done')